In [14]:
import geopandas as gpd
import sv_config as sc
import os
import time

dirname = os.path.abspath('')

gpkg_input = os.path.join(dirname, sc.geopackagePath)
gdf_samplepoint = gpd.read_file(gpkg_input,
                                layer=sc.samplePointsData_withoutNan)
gdf_hex = gpd.read_file(gpkg_input, layer=sc.hex250)
# output geopackage
gpkg_output = os.path.join(dirname, sc.output_hex_250m)

# calculate all the "urban_sample_point_count" for each hex
samplepoint_count = gdf_samplepoint['hex_id'].value_counts()

gdf_hex_new = gdf_hex.join(samplepoint_count, how='inner', on='index')

gdf_hex_new.rename(columns={'hex_id': 'urban_sample_point_count'},
                   inplace=True)


gdf_samplepoint.columns

def aggregation(gdf_hex, gdf_samplePoint, fieldNames):
    """
    calculate aggregation for hex
    
    Arguments:
        gdf_hex {geopandas} -- [hex]
        gdf_samplePoint {geopandas} -- [sample point]
        fieldNames {list(zip)} -- [fieldNames from sample point and hex ]
    """
    for names in fieldNames:
        df = gdf_samplePoint[['hex_id', names[0]]].groupby('hex_id').mean()
        gdf_hex = gdf_hex.join(df, how='left', on='index')
        gdf_hex.rename(columns={names[0]: names[1]}, inplace=True)
    return gdf_hex

fieldNames_from_samplePoint = [
    'sp_nearest_node_Supermarket_binary', 'sp_nearest_node_Convenience_binary',
    'sp_nearest_node_Public transport stop (any)_binary',
    'sp_nearest_node_aos_nodes_30m_line_binary', 'sp_local_nh_avg_pop_density',
    'sp_local_nh_avg_intersection_density', 'sp_daily_living_score',
    'sp_walkability_index'
]

fieldNames2hex = [
    'pct_access_500m_supermarkets', 'pct_access_500m_convenience',
    'pct_access_500m_pt_any', 'pct_access_500m_public_open_space',
    'local_nh_population_density', 'local_nh_intersection_density',
    'local_daily_living', 'local_walkability'
]

gdf_hex_new = aggregation(gdf_hex_new, gdf_samplepoint,list(zip(fieldNames_from_samplePoint, fieldNames2hex)))

In [15]:
fields = [
    'pct_access_500m_supermarkets', 'pct_access_500m_convenience',
    'pct_access_500m_pt_any', 'pct_access_500m_public_open_space'
]
gdf_hex_new[fields] = gdf_hex_new[fields]*100

In [16]:
hex_fieldNames = {
    'index': 'index',
    'study_region': 'study_region',
    'urban_sample_point_count': 'urban_sample_point_count',
    'pct_access_500m_supermarkets': 'pct_access_500m_supermarkets',
    'pct_access_500m_convenience': 'pct_access_500m_convenience',
    'pct_access_500m_pt_any': 'pct_access_500m_pt_any',
    'pct_access_500m_public_open_space': 'pct_access_500m_public_open_space',
    'local_nh_population_density': 'local_nh_population_density',
    'local_nh_intersection_density': 'local_nh_intersection_density',
    'local_daily_living': 'local_daily_living',
    'local_walkability': 'local_walkability',
    'all_cities_z_nh_population_density': 'all_cities_z_nh_population_density',
    'all_cities_z_nh_intersection_density':
    'all_cities_z_nh_intersection_density',
    'all_cities_z_daily_living': 'all_cities_z_daily_living',
    'all_cities_walkability': 'all_cities_walkability',
    'geom': 'geom'
}


In [17]:
def organiseColumnName(gdf, fieldNames):
    """organise the gdf column name to make it match the desired result
    note: at this stage, some of the fields haven't had number
    Arguments:
        gdf {geodataframe} -- [the hex should be rename the column]
        fieldNames {[list]} -- [all the desired field names]
    """
    fieldNamesOld = gdf.columns.to_list()
    fields = []
    for i in fieldNamesOld:
        if i in fieldNames:
            fields.append(i)
    return gdf[fields].copy()

In [19]:
fieldNames = list(hex_fieldNames.values())
gdf_hex_new = organiseColumnName(gdf_hex_new, fieldNames)

In [20]:
gdf_hex_new

,index,urban_sample_point_count,pct_access_500m_supermarkets,pct_access_500m_convenience,pct_access_500m_pt_any,pct_access_500m_public_open_space,local_nh_population_density,local_nh_intersection_density,local_daily_living,local_walkability
1,11077,37,64.864865,70.270270,89.189189,97.297297,4955.353575,219.610385,3.216216,-1.360327
2,11078,9,0.000000,88.888889,88.888889,100.000000,4955.353575,219.610385,2.777778,-2.271641
3,11079,19,73.684211,100.000000,100.000000,100.000000,4955.353575,219.610385,3.736842,-0.278183
4,11261,7,100.000000,100.000000,100.000000,100.000000,4955.353575,219.610385,4.000000,0.268803
5,11262,30,100.000000,100.000000,100.000000,100.000000,4955.353575,219.610385,4.000000,0.268803
6,11263,54,100.000000,100.000000,100.000000,100.000000,4955.353575,219.610385,4.000000,0.268803
7,11264,42,54.761905,95.238095,100.000000,100.000000,4955.353575,219.610385,3.500000,-0.770469
8,11265,47,91.489362,100.000000,100.000000,100.000000,4955.353575,219.610385,3.914894,0.091905
10,11448,10,100.000000,100.000000,100.000000,100.000000,4955.353575,219.610385,4.000000,0.268803
11,11449,42,100.000000,100.000000,100.000000,100.000000,4955.353575,219.610385,4.000000,0.268803
